In [192]:
!pip install scikit-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 3.9 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Using cached joblib-1.3.1-py3-none-any.whl (301 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils c

In [1]:
import wikipediaapi

import os
import openai
import json
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
openai.api_key = json.load(open("/Users/shahules/openai-key.json"))['ikka']

/opt/anaconda3/envs/alerts/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def llm(messages, **kwargs):
    response = openai.ChatCompletion.create(
        model=kwargs.get("model", "gpt-3.5-turbo"),
        messages=messages,
        temperature=kwargs.get("temperature", 0),
        top_p=kwargs.get("top_p", 1),
        frequency_penalty=kwargs.get("frequency_penalty", 0.0),
        presence_penalty=kwargs.get("presence_penalty", 0.0),
        max_tokens=kwargs.get("max_tokens", 500),
        n=kwargs.get("n", 1),
    )
    return response


In [3]:
message = [{"role": "system", "content": "You're a bot that answers any given question. If you dont know the exact answer make up one."},
{"role":"user", "content":"What were the temperatures and snowfall amounts during the cold snap in Afghanistan in January 2023, and how many people and livestock were affected?"}]

In [4]:
# llm(message)

In [5]:
wiki_wiki = wikipediaapi.Wikipedia(
    language="en", extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("Black hole")


def get_page_section(page,chars=8000):
    all_text = ""
    p_wiki = wiki_wiki.page(page)    
    return p_wiki.text[:chars]


def get_cosine(page, backlinks):
    backlinks_vec = model.encode(backlinks)
    page_vec = model.encode([page]).reshape(1,-1)
    norm = np.linalg.norm(backlinks_vec,axis=1)*np.linalg.norm(page_vec,axis=1)
    cosine_sim = np.dot(backlinks_vec,page_vec.T).reshape(-1,)/norm
    return cosine_sim

def get_backlink_titles(page):
    p_wiki = wiki_wiki.page(page) 
    backlinks =  [i.title() for i in p_wiki.backlinks if ":" not in i.title()][:100]
    if len(backlinks)>1:
        c = get_cosine(page, backlinks)
        top_indices = c.argsort()[::-1][:10]
        return [backlinks[i] for i in top_indices if backlinks[i]!=page]
    return []
    
    

In [6]:
Question_gen = """
Given a wikipedia text generate a question that can be fully answered from the text of medium difficulty with long form answer seperated by \n\n from the given text.

Wikipedia Text:
The Eiffel Tower is a wrought-iron lattice tower located on the Champ de Mars in Paris, France. It was named after the engineer Gustave Eiffel, whose company designed and built the structure. Erected in 1889 as the entrance arch to the 1889 World's Fair, it has become a global cultural icon of France and one of the most recognizable structures in the world. The Eiffel Tower is the tallest structure in Paris and the most-visited paid monument in the world; millions of people ascend it every year. The tower stands at a height of 324 meters (1,063 feet) and was the tallest man-made structure in the world until the completion of the Chrysler Building in New York City in 1930.
The tower has three levels for visitors, with restaurants on the first and second levels. The third level observatory's upper platform is at 276 meters (906 feet) above the ground, the highest accessible to the public in the European Union. The tower has been featured in numerous films and TV shows, and its lighting is often modified to mark special events or holidays. Despite initial criticism from some of France's leading artists and intellectuals, the Eiffel Tower has become a global cultural icon of France and one of the most recognizable structures in the world.
Question: What is the height of the Eiffel Tower, and how does it compare to the height of the Chrysler Building in New York City?
\n\nAnswer:The height of the Eiffel Tower is 324 meters (1,063 feet). It was the tallest man-made structure in the world until the completion of the Chrysler Building in New York City in 1930. The Chrysler Building surpassed the Eiffel Tower's height and became the new tallest structure, reaching a height of 319 meters (1,046 feet) including its spire. However, it's important to note that the Eiffel Tower remains the tallest structure in Paris and still holds its title as the most-visited paid monument in the world. Despite losing its status as the tallest man-made structure globally, the Eiffel Tower's cultural significance and iconic status endure, attracting millions of visitors each year and leaving an indelible mark on the city of Paris and the world at large.
Wikipedia Text:\n{}
"""

In [7]:
from tqdm import tqdm
def generate_dataset(pages):
    data = []
    for page in tqdm(pages):
        try:
            text = get_page_section(page)
            message = [{"role": "user", "content":Question_gen.format(text)}]
            output = llm(message)
            question, answer_grounded = output['choices'][0]['message']['content'].split("\n\n")
            message = [{"role": "user", "content":question}]
            answer = llm(message)['choices'][0]['message']['content']
            data.append({
                "question":question,
                "grounded_answer":answer_grounded,
                "answer":answer,
                "context":text,
                "source":page,
            })
        except Exception as e:
            print(e)
    return data
        
        

https://en.wikipedia.org/wiki/2023#January

In [27]:
with open("/Users/shahules/belar/experimental/pages.txt", "r") as file:
    pages = file.read()


In [28]:
# output = llm(Question_gen.format(c.text))

In [29]:
# question, answer = output['choices'][0]['message']['content'].split("\n\n")

In [30]:
pages = pages.split('\n')[:-1]

In [31]:
pages = ["Volcanism on Venus","Pandemic prevention","Jupiter Icy Moons Explorer"]

In [32]:
data_sample = generate_dataset(pages)

100%|█████████████████████████████████████████████████████| 3/3 [00:54<00:00, 18.14s/it]


In [34]:
data = json.load(open("/Users/shahules/belar/experimental/ragas_wiki_eval.json"))
data.extend(data_sample)

In [35]:
len(data)

50

In [36]:
with open("ragas-eval-data.json",'w') as file:
    json.dump(data, file, indent=4)

In [50]:
"""
    get_recent_changes.py

    MediaWiki API Demos
    Demo of `RecentChanges` module: Get the three most recent changes with
    sizes and flags

    MIT License
"""

import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "format": "json",
    "rcprop": "title|timestamp|tags",
    "list": "recentchanges",
    "action": "query",
    "rclimit": "1000",
    "rctype":"new"
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

RECENTCHANGES = DATA['query']['recentchanges']
remove_tags = ["User:","Category:","Draft","User talk:","Talk:","Wikipedia:","Template talk:"]

for rc in RECENTCHANGES:
    title = str(rc['title'])
    timestamp = str(rc['timestamp'])
    if not any([x in title for x in remove_tags]) and "2021" not in timestamp:
        
        print(title)


Sorn-khiri Sriprachuap
Methane synthesis
1993–94 BCFC season
Hagawi
Populus mexicana
Gracie Lawrence
Gracie Bea Lawrence
Laura Cornelius
A. D. Macklin
Polyosma hirsuta
Maltzanella
Molina Seca
The Life of the Party (play)
Ramnagar Assembly constituency (disambiguation)
Blue Angel (Park Ji-yoon album
A.D. Macklin
Blanca Wiethüchter López
Frances Hughes
Gubad Ibadoghlu
2023 BWF World Championships – Men's doubles
Jesús Urzagasti Aguilera
1925 Reform Party (New Zealand) leadership election
Ana Pinho Rodrigues
Olympian 6
80th Venice Film Festival
2023 Venice Film Festival
2023 Venice International Film Festival
We Find the Bunyip
C-ya-laterrrr
Evan asano
13th Politburo Standing Committee of the Chinese Communist Party
Chase Blasi
Hateful speech
Template:WIR-280
Archaeology in Africa
Home (Secret Invasion)
Anil Thadani
Love Me a Little
Wayne Stevens (disambiguation)
Wayne Osborne (disambiguation)
Wayne Hughes (disambiguation)
Wayne Hammond (disambiguation)
Thai Empire (disambiguation)
Oxygen

In [286]:
data = json.load(open("ragas-data.json"))

In [94]:
evading = ["I'm sorry", "as an AI language model"]

In [99]:
topics_regen = [item for item in data if any(x.lower() in item['answer'].lower() for x in evading)]

In [101]:
message = [{"role": "system", "content": "You're a bot that answers any given question. If you dont know the exact answer make up one."},]

In [105]:
for item in topics_regen:
    message = [{"role": "system", "content": "You're a bot that answers any given question. If you dont know the exact answer make up one."},]
    message.append({"role":"user","content":item['question']})
    output = llm(message)
    item['answer'] = output['choices'][0]['message']['content']

In [42]:
for item in tqdm(data):
    titles = get_backlink_titles(item["source"])
    context = [get_page_section(item,chars=1000) for item in titles[:2]]
    context.insert(0,item['context'])
    item["context_retrieved"] = context

100%|███████████████████████████████████████████████████| 50/50 [01:23<00:00,  1.67s/it]


In [119]:
with open("ragas_wiki_evalv1.json",'w') as file:
    json.dump(data, file, indent=4)

In [56]:
Dataset.from_json("ragas-eval-data.json").push_to_hub("explodinggradients/wiki-eval")

Found cached dataset json (/Users/shahules/.cache/huggingface/datasets/json/default-d425d908fb97025a/0.0.0)
Creating parquet from Arrow format: 100%|█████████████████| 1/1 [00:00<00:00, 50.02ba/s]

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:06<00:00,  6.27s/it]


## Generate low relevancy answers

In [10]:
from datasets import load_dataset

In [22]:
irr_answer = """
Answer the given question partially.
Question:{}
"""

In [12]:
dataset = load_dataset("explodinggradients/wiki-eval")

Extracting data files: 100%|████████| 1/1 [00:00<00:00, 389.52it/s]


Dataset parquet downloaded and prepared to /Users/shahules/.cache/huggingface/datasets/explodinggradients___parquet/explodinggradients--wiki-eval-f0df84e235efd078/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|███████████████████████████████| 1/1 [00:00<00:00, 224.16it/s]


In [29]:
answers = []
for item in dataset['train']:
    question = item['question']
    message = []
    message.append({"role":"user","content":irr_answer.format(question)})
    while True:
        try:
            answer = llm(message)['choices'][0]['message']['content']
        except Exception as e:
            continue
        break
    answers.append(answer)


In [31]:
dataset = dataset["train"].add_column("partial_answer",answers)

In [32]:
dataset.push_to_hub("explodinggradients/wiki-eval")

Pushing dataset shards to the dataset hub:   0%| | 0/1 [00:00<?, ?i
Creating parquet from Arrow format: 100%|█| 1/1 [00:00<00:00, 79.01

Upload 1 LFS files: 100%|████████████| 1/1 [00:06<00:00,  6.60s/it]
Pushing dataset shards to the dataset hub: 100%|█| 1/1 [00:07<00:00
Deleting unused files from dataset repository: 100%|█| 1/1 [00:00<0
Updating downloaded metadata with the new split.


In [231]:
with open("ragas_wiki_evalv1.json",'w') as file:
    json.dump(data, file, indent=4)